# Test of whether LanguageModelLoader deliver batch where each row is a continuous set of tokens


Status is that the test works with

-MyLanguageModelLoader 

-fastai's LanguageModelLoader, however, throws an exception while indexing the jagged array


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import * 
import numpy as np
from languagemodelloader import *

In [3]:
def getAllBatches(data,epochs=1):
    x=None
    for i in range(epochs):
        for xb,yb in data:
            d= xb.data.numpy()
            if x is None: 
                x = xb.data.numpy().copy()
            else:         
                x = np.concatenate((x, xb.data.numpy().copy()),axis=1)
            continue
    return x            

In [4]:
def generateContinuousNumber(bs,seq_len,iterations,minTokens):
    "The data consist of continuously increasing numbers i jagged array with randomly varying layout"
    jagged = []
    countTokens = 0
    totalTokens = bs*seq_len*iterations
    #print(f"totalTokens:{totalTokens}")
    rand_interval=20
    while countTokens < totalTokens:
        nb = totalTokens-countTokens if   totalTokens-countTokens <rand_interval\
                                     else minTokens+int(np.random.random() * rand_interval)
        jagged.append(np.arange(countTokens+1,countTokens+1+nb))
        countTokens = jagged[-1][-1]
    jagged = np.asarray(jagged)    
    return jagged, countTokens

def printJagged(jagged, countTokens):
    print(f"countTokens:{countTokens}")
    for j in jagged: print(j)

In [15]:
def test_datadirection( bs,seq_len,iterations,minTokens, forward=True, nbInoutGenerations=10, log=False):
    for i in range(nbInoutGenerations):
        jagged,countTokens = generateContinuousNumber(bs,seq_len,iterations,minTokens)
        if log: printJagged(jagged, countTokens)
            
        trainIDS = validIDS = jagged
        db = TextLMDataBunch.from_ids( ".", None, trainIDS, validIDS, bptt=seq_len, p_bptt=p_bptt, bs=bs)

        data = MyLanguageModelLoader(dataset=db.train_dl, bptt=seq_len, p_bptt=p_bptt, bs=bs, shuffle=False, 
                                     backwards=not forward, log=log)
        batches = getAllBatches(data)
        if log: display(pd.DataFrame(data=batches))

        #The diff from one to the next column must be 1 for aligned mini-batches with forward indexing of the data
        #(forward is default for LanguageModelLoader ie.: backwards=False) 
        b_diff = batches[:,1:] - batches[:,0:-1]
        if log: display(pd.DataFrame(data=b_diff))
            
        diff_value = 1 if forward else -1
        assert (b_diff.flatten()==diff_value).all()

# Test continouity of tokens in batches loaded forwards and backwards

In [6]:
#b_ptt     = 0 for MyLanguageModelLoader
p_bptt     =  0  #deactivates changes in the sequence length in MyLanguageModelLoader so that we do not wrap around

#create test data so that we can control whether the LanguageModelLoader returns contigous tokens 
#The genrated data must be size so that the batches will not wrap aoround. 
bs         =  8
seq_len    =  3  #=bptt
iterations =  2
minTokens  =  1 #in a rag array

In [13]:
%timeit test_datadirection( bs, seq_len, iterations, minTokens, forward=True, nbInoutGenerations=200)

118 ms ± 7.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit test_datadirection( bs, seq_len, iterations, minTokens, forward=False, nbInoutGenerations=1, log=True)

countTokens:48
[1 2 3 4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23]
[24 25 26 27 28 29 30 31 32 33]
[34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
stepTokens:6.0
i_rag:0 ln_rag:15 int(stepTokens * i):0 countTokens:0 self.eo[i]:15 
i_rag:0 ln_rag:15 int(stepTokens * i):6 countTokens:0 self.eo[i]:9 
i_rag:0 ln_rag:15 int(stepTokens * i):12 countTokens:0 self.eo[i]:3 
i_rag:1 ln_rag:10 int(stepTokens * i):18 countTokens:15 self.eo[i]:7 
i_rag:1 ln_rag:10 int(stepTokens * i):24 countTokens:15 self.eo[i]:1 
i_rag:2 ln_rag:9 int(stepTokens * i):30 countTokens:25 self.eo[i]:4 
i_rag:3 ln_rag:10 int(stepTokens * i):36 countTokens:34 self.eo[i]:8 
i_rag:3 ln_rag:10 int(stepTokens * i):42 countTokens:34 self.eo[i]:2 
BEGIN:ei:0 i:4 eo:15 len(rag):15 row.size:4 ibuf:0 bi:0 bo:15 first toke:48
ITE:  ei:0 i:4 eo:15 len(rag):15 row.size:4 ibuf:0 n:4 bi:0 bo:15 last toke:44
ENDB:ei:ei:0 i:4 eo:12 len(rag):15 row.size:4 ibuf:4 n:4 bi:0 bo:15 ENDB2:last toke:45
BEGIN:ei:0 i:4 eo:9 len

,0,1,2,3,4,5
0,48,47,46,45,44,43
1,42,41,40,39,38,37
2,36,35,34,43,42,41
3,30,29,28,27,26,25
4,24,23,22,21,20,19
5,18,17,16,15,14,13
6,12,11,10,9,8,7
7,6,5,4,3,2,1


,0,1,2,3,4
0,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1
2,-1,-1,9,-1,-1
3,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1


AssertionError: 